## Create FHIR R4 SearchParameter Resource

Create FHIR R4 SearchParameter Resource, Quick start text, and Searchparameter list using the python fhir client

Source data is in excel file

### Prerequisites:

- Python 3.7 or greater

### Import FHIRR4Client and other libraries

In [42]:
%config IPCompleter.greedy=True

In [43]:
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from fhirclient.r4models import searchparameter as SP
from fhirclient.r4models import capabilitystatement as CS
from fhirclient.r4models import bundle as B
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from IPython.display import display as Display, HTML, Markdown
from pprint import pprint
from collections import namedtuple
from pandas import *
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape
import R4sp_summary_list as sp_map
from stringcase import snakecase, titlecase, pascalcase
from itertools import zip_longest

####  Assign Global Variables


Here is where we assign all the global variables for this example such as the local paths for file input and output

##### Need to update:
- base_id
- paths
- canonical

In [44]:
#******************** Need to update *************************************************
fhir_base_url = 'http://hl7.org/fhir/'
base_id = "US-Core-R4"
canon_base = "http://hl7.org/fhir/us/core/"
ig_folder = 'US-Core'
publisher = 'HL7 International - Structured Documents Work Group'
publisher_endpoint = dict(
                        system = 'url',
                        value = 'http://www.hl7.org/Special/committees/structure/index.cfm'
                        ) 

#***********************************************************************************

md_template = ['search_narrative.j2', 'sp_list_page.j2', 'cs_search_documentation.j2']


fhir_term_server = 'http://test.fhir.org/r3'

# profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient



none_list = ['', ' ', 'none', 'n/a', 'N/A', 'N', 'False', 'FALSE']
sep_list = (',', ';', ' ', ', ', '; ')



### validate

In [45]:
# *********************** validate Resource as Dict ********************************

def validate(r):

    fhir_test_server = 'http://test.fhir.org/r4'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    
    params = {
    }
    
    r = post(f'{fhir_test_server}/{r["resourceType"]}/$validate', params = params, headers = headers, data = dumps(r))
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### Write to File

In [46]:
 def write_file(name, data): # write file
    
    out_path = ''
    #out_path = f'//ERICS-AIR-2/ehaas/Documents/FHIR/{ig_folder}/source/resources/'
    with open(f'{Path(out_path)}/{name}.json', 'w') as f:
        f.write(data)

### Get Search Parameter input data

In [47]:
#in_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
in_path =''

in_file ="uscore-server"

xls = ExcelFile(f'{in_path}{in_file}.xlsx')
df = read_excel(xls,'sps',na_filter = False)

df

,base,base_conf,code,exists,update,XBase,type,expression,multipleOr,multipleOr_conf,...,comparator,comparator_conf,references,chain,chain_conf,combo_pairs,combo_conf,description,example_query,imp_note
0,Questionnaire,SHALL,_id,Y,N,Resource,token,Questionnaire.id,N,,...,,,,,,,,n/a,n/a,foo
1,Questionnaire,SHALL,url,Y,N,Questionnaire,uri,Questionnaire.url,N,,...,n/a,,n/a,n/a,,,,n/a,n/a,
2,Questionnaire,SHALL,status,Y,Y,Questionnaire,token,Questionnaire.status,Y,,...,n/a,,n/a,n/a,,"title,publisher,version,context-type-value",SHALL,n/a,n/a,
3,Questionnaire,SHALL,title,Y,Y,Questionnaire,string,Questionnaire.title,Y,,...,n/a,,n/a,n/a,,version,SHOULD,n/a,n/a,
4,Questionnaire,SHALL,publisher,Y,Y,Questionnaire,string,Questionnaire.publisher,Y,,...,n/a,,n/a,n/a,,"context-type-value,version",SHOULD,n/a,n/a,
5,Questionnaire,SHOULD,version,Y,Y,Questionnaire,token,Questionnaire.version,Y,,...,n/a,,n/a,n/a,,,,n/a,n/a,
6,Questionnaire,SHOULD,context-type-value,N,N,Questionnaire,composite,Questionnaire.context-type-value,N,,...,n/a,,n/a,n/a,,,,n/a,n/a,
7,QuestionnaireResponse,SHALL,_id,Y,N,Resource,token,QuestionnaireResponse.id,N,,...,n/a,,n/a,n/a,,,,n/a,n/a,
8,QuestionnaireResponse,SHALL,questionnaire,Y,Y,QuestionnaireResponse,reference,QuestionnaireResponse.questionnaire,Y,,...,n/a,,n/a,n/a,,,,n/a,n/a,
9,QuestionnaireResponse,SHALL,patient,Y,Y,QuestionnaireResponse,reference,QuestionnaireResponse.patient,Y,,...,n/a,,n/a,"patient.gender,patient.birthdate,patient.race,...","SHALL,SHALL,SHALL,SHALL,SHALL","author,source,context",SHALL,n/a,n/a,


In [48]:
data = [i for i in df.itertuples(index=True)]
   
for d in data:
    print(f'Rresource = {d.base}, Search Parameter = {d.code}, Exists = {d.exists}')

Rresource = Questionnaire, Search Parameter = _id, Exists = Y
Rresource = Questionnaire, Search Parameter = url, Exists = Y
Rresource = Questionnaire, Search Parameter = status, Exists = Y
Rresource = Questionnaire, Search Parameter = title, Exists = Y
Rresource = Questionnaire, Search Parameter = publisher, Exists = Y
Rresource = Questionnaire, Search Parameter = version, Exists = Y
Rresource = Questionnaire, Search Parameter = context-type-value, Exists = N
Rresource = QuestionnaireResponse, Search Parameter = _id, Exists = Y
Rresource = QuestionnaireResponse, Search Parameter = questionnaire, Exists = Y
Rresource = QuestionnaireResponse, Search Parameter = patient, Exists = Y
Rresource = QuestionnaireResponse, Search Parameter = context, Exists = Y
Rresource = QuestionnaireResponse, Search Parameter = status, Exists = Y
Rresource = QuestionnaireResponse, Search Parameter = author, Exists = Y
Rresource = QuestionnaireResponse, Search Parameter = source, Exists = Y
Rresource = Patient

### update core SP with additional capabiliities


- Get definitions bundle and convert to python object for ease of notation
- use sp_map to map to Type + parameter
- If need to update SP Extract the SP based on the excel file

### load SP Mapping dictionary

In [49]:
p = Path('C:/Users/Eric/Documents/HL7/FHIR/BUILD_EDIT_FILES/R4_Definitions/search-parameters.json')

b = B.Bundle(loads(p.read_text()), strict = False)
sp = b.entry[0]
print(sp.fullUrl)
pprint([i for i in sp_map.sp_list2 if i[0]=="Observation" and i[1] == 'code'])
sp.as_json()

http://hl7.org/fhir/SearchParameter/DomainResource-text
[('Observation', 'code', 'http://hl7.org/fhir/SearchParameter/clinical-code')]


{'fullUrl': 'http://hl7.org/fhir/SearchParameter/DomainResource-text',
 'resource': {'id': 'DomainResource-text',
  'extension': [{'url': 'http://hl7.org/fhir/StructureDefinition/structuredefinition-standards-status',
    'valueCode': 'trial-use'}],
  'base': ['DomainResource'],
  'code': '_text',
  'contact': [{'telecom': [{'system': 'url', 'value': 'http://hl7.org/fhir'}]},
   {'telecom': [{'system': 'url',
      'value': 'http://www.hl7.org/Special/committees/fiwg/index.cfm'}]}],
  'date': '2019-04-18T15:23:58+00:00',
  'description': 'Search on the narrative of the resource',
  'experimental': False,
  'modifier': ['missing', 'exact', 'contains'],
  'multipleAnd': True,
  'multipleOr': True,
  'name': '_text',
  'publisher': 'Health Level Seven International (FHIR Infrastructure)',
  'status': 'draft',
  'type': 'string',
  'url': 'http://hl7.org/fhir/SearchParameter/DomainResource-text',
  'version': '4.1.0',
  'xpathUsage': 'normal',
  'resourceType': 'SearchParameter'}}

## create updated SPs
- in spreadsheet sp.elements as comma separated list and sp/element_conf as comma separated list
- if not listed then conformance is MAY 

In [50]:
mults = [
'multipleOr',
'multipleAnd',
]

mods =  [   
'modifier',
'comparator',
]

def sp_expectation(conf=None):
    if not conf:
        conf = ["MAY"]
    x_list=[]
    print(len(conf))
    for i in conf:
        x = X.Extension(dict(
        url = f'http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation',
        valueCode = i if i else "MAY"
        ))
        x_dict = dict(
        extension = [x.as_json()]
        )
        x_list.append(x_dict)
    
    return x_list


sp_list=[]
for d in data:
    for i in b.entry:
        sp = i.resource
        if d.update =='Y' and d.base in sp.base and d.code == sp.code: #update sp
            # print(dumps(i.resource.as_json(),indent = 3))
            # change id and url, publisher, and contact, draft etc
            sp.derivedFrom =sp.url
            sp.url = f'{canon_base}SearchParameter/{sp.id}'
            sp.publisher = publisher
            sp.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
            sp.date = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
            sp.name = pascalcase(sp.id.replace('-','_'))
            sp.status = 'active'
            #convert to dict since model can't handle primitive extensions
            sp_dict = sp.as_json()
            for e in mults:
                sp_dict[e] = False if getattr(d,e) in none_list else True
                sp_dict[f'_{e}'] = sp_expectation(conf=list(getattr(d,f'{e}_conf')))[0]
    
            for e in mods:
                v_list = getattr(d,e).split(',')
                c_list = getattr(d,f'{e}_conf').split(',')
                z_list = list(zip(v_list,c_list))
                #print(z_list)
                try:
                    c_list = [(z_list[0][1] if z_list[0][0] == v else None) for v in sp_dict[e]]
                    print(c_list)
                    sp_dict[f'_{e}'] = sp_expectation(conf=c_list)
                except KeyError:
                    pass
            if d.chain not in none_list:
                sp_dict['chain'] = d.chain.split(',')
                sp_dict['_chain'] = sp_expectation(conf=d.chain_conf.split(',')) 
            
            print(f'======================= SP = {sp_dict["id"]} =====================')
            print(dumps(sp_dict,indent=4))
            sp_list.append(sp_dict)

1
1
[None, None, None, None, None, None, None, None]
8
======================= SP = Questionnaire-status =====================
{
    "id": "Questionnaire-status",
    "extension": [
        {
            "url": "http://hl7.org/fhir/StructureDefinition/structuredefinition-standards-status",
            "valueCode": "trial-use"
        }
    ],
    "base": [
        "Questionnaire"
    ],
    "code": "status",
    "contact": [
        {
            "telecom": [
                {
                    "system": "url",
                    "value": "http://www.hl7.org/Special/committees/structure/index.cfm"
                }
            ]
        }
    ],
    "date": "2019-04-20T00:27:41.244471Z",
    "derivedFrom": "http://hl7.org/fhir/SearchParameter/Questionnaire-status",
    "description": "The current status of the questionnaire",
    "experimental": false,
    "expression": "Questionnaire.status",
    "modifier": [
        "missing",
        "text",
        "not",
        "in",
        

### Validate

In [51]:
for i in sp_list:
    
    r = validate(i)
    display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))               

### Create Markdown Text for Search.

- Using Jinja2 Template create markdown file for search section in profiles profiles

### define utility to convert json string to python object

In [52]:
class d2o(object):
    def __init__(self, d):
        self.__dict__ = d

In [148]:
r_type = ['Questionnaire', 'QuestionnaireResponse', 'Patient']  # TODO  loop over all types in sp_list


search_type = dict(
    token = '{[system]}|[code]',
    id = '[id]',
    reference = '[url]',
    string = '[string]'
    )

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

template = env.get_template(md_template[0])

# use sp_dict to create the narratvie using Jinja2
'''1. **{{i.base_conf}}** support search by the **`{{i.code}}`** parameter:

  `GET [base]/{{i.base[0]}}?{{i.code}}={{search_type[i.type]}}`
{%- endfor %}
for t in r_type:
    sp = [d2{%- endfor %}o(i) for i in sp_list if i['base'][0] in  r_type]
'''

### Use Spreadsheet to create the narrative using Jinja2 Templates
#print(data[0])
for d in data:
    for i,c in enumerate(d.modifier_conf.split(',')):
        if c =="SHALL":
            d.apply('foo') = 'foo'
            print(d)

        
for t in r_type:  
    sp = [d for d in data if d.base in t]
    #print(sp)
    #print(sp[0].base, sp[0].base_conf, sp[0].modifier, sp[0].modifier.split(','))
    
    
    
    #search_md = template.render(sp=sp,search_type=search_type)

    #display(Markdown(search_md))


SyntaxError: can't assign to function call (<ipython-input-148-57b29e94c0da>, line 32)

### Create Markdown Text for SearchParameters Page

- Using Jinja2 Template create markdown file for searchparameters page

In [96]:
# get types:
r_list = []
for i in sp_list:
    for j in i.base:
        r_list.append(j)

r_list = sorted(set(r_list))

# md_template = 'sp_list_page.j2'

template = env.get_template(md_template[1])
display(Markdown(template.render(sp_list=sp_list,r_list=r_list)))
searchparameters_md = template.render(sp_list=sp_list,r_list=r_list)


AttributeError: 'dict' object has no attribute 'base'